# Package Functionalities Demo:

## Preparation: load data, functions, libraries of dependency...

In [1]:
pwd()

"/Users/FredYu/Documents/GitHub/BulkLMM.jl"

In [2]:
cd("test")

In [3]:
include("BXDdata_for_test.jl");
include("../src/parallel_helpers.jl");
include("../src/scan_LiteQTL.jl");
include("testHelpers.jl");

LoadError: LoadError: UndefVarError: @tturbo not defined
in expression starting at /Users/FredYu/Documents/GitHub/BulkLMM.jl/src/scan_LiteQTL.jl:259
in expression starting at /Users/FredYu/Documents/GitHub/BulkLMM.jl/src/scan_LiteQTL.jl:255

In [4]:
using BenchmarkTools
using LoopVectorization

In [5]:
traitID = 7919;
pheno_y = reshape(pheno[:, traitID], :, 1);

## 1. Simple Univariate Scans:

Given the raw phenotype info of one trait, the genotype info and the corresponding pre-calculated kinship matrix, return the lod scores of all genetic markers on that particular trait.

In [6]:
?scan

search: scan scan_alt scan_null scan_lite scan_perms scan_perms_lite



```
scan(y, g, K, reml, method)
```

Performs genome scan for univariate trait and each of the gene markers, one marker at a time (one-df test) 

# Arguments

  * y = 1d array of floats consisting of the N observations for a certain quantitative trait (dimension: N*1)
  * g = 2d array of floats consisting of all p gene markers (dimension: N*p)
  * K = 2d array of floats consisting of the genetic relatedness of the N observations (dimension:N*N)
  * prior_a = a float of prior distribution parameter
  * prior_b = a float of prior distribution parameter

# Keyword arguments

  * addIntercept = Boolean flag indicating if an intercept column needs to be added to the design matrix; Default is to add an intercept
  * reml = Bool flag indicating if VCs are estimated by REML likelihood; Default is ML
  * assumption = String indicating whether the variance component parameters are the same across all markers (null) or not (alt); Default is `null`
  * method = String indicating the matrix factorization method to use; Default is QR.

# Value

A list of output values are returned:

  * out00.sigma2 = Float; estimated marginal variance due to random errors (by null lmm)
  * out00.h2 = Float; estimated heritability (by null lmm)
  * lod = 1d array of floats consisting of the lod scores of this trait and all markers (dimension: p*1)

# Some notes

```
This function calls either `scan_null` or `scan_alt` depending on the input passed as `method`.
Output data structure might need some revisions.
```


### Assumption (1): VCs are estimated once from the null model

In [7]:
?scan_null

search: scan_null



```
scan_null(y, g, K, reml, method)
```

Performs genome scan for univariate trait and each of the gene markers, one marker at a time,  assuming the variance components are the same for all markers.

# Arguments

  * y = 1d array of floats consisting of the N observations for a certain quantitative trait (dimension: N*1)
  * g = 2d array of floats consisting of all p gene markers (dimension: N*p)
  * K = 2d array of floats consisting of the genetic relatedness of the N observations (dimension:N*N)

# Keyword arguments

  * addIntercept = Boolean flag indicating if an intercept column needs to be added to the design matrix; Default is to add an intercept
  * reml = Bool flag indicating if VCs are estimated by REML likelihood; Default is ML
  * method = String indicating the matrix factorization method to use; Default is QR.

# Value

A list of output values are returned:

  * out00.sigma2 = Float; estimated marginal variance due to random errors (by null lmm)
  * out00.h2 = Float; estimated heritability (by null lmm)
  * lod = 1d array of floats consisting of the lod scores of this trait and all markers (dimension: p*1)

# Some notes

```
This is a subsequent function that does univariate scan. The variance components are estimated once 
and used for all markers. To be called by the `scan` function when the `method = ` field is passed 
as `null` (default).
```


In [8]:
@time test_null = scan(pheno_y, geno, kinship; addIntercept = true, reml = false, assumption = "null", method = "cholesky");

  0.042875 seconds (73.52 k allocations: 33.189 MiB)


In [9]:
test_null[1] # sigma2_e

0.04983520992588347

In [10]:
test_null[2] # hsq

0.8960637228416714

In [11]:
test_null[3] # LODs

7321-element Vector{Float64}:
 0.008196360329322583
 0.008196360329322583
 0.008196360329322583
 0.008196360329322583
 0.008196360329322583
 0.008196360329322583
 0.008196360329322583
 0.008196360329322583
 0.008196360329322583
 0.008196360329322583
 0.008196360229954236
 0.008195284184264295
 0.00987347031983854
 ⋮
 0.09285575916070832
 0.09285575883025671
 0.09285575883025671
 0.011173043848082764
 0.008215696058314959
 0.008215696058314959
 0.01502332773404691
 0.02895771332730057
 0.02731129976924651
 0.04498777747884275
 0.012828276502853786
 0.012828276502853786

### Assumption (2): VCs are re-estimated everytime for each marker scan

In [12]:
?scan_alt

search: scan_alt scan_lite_univar



```
scan_alt(y, g, K, reml)
```

Performs genome scan for univariate trait and each of the gene markers, one marker at a time (one-df test), assuming the variance components may not be the same for all markers. 

# Arguments

  * y = 1d array of floats consisting of the N observations for a certain quantitative trait (dimension: N*1)
  * g = 2d array of floats consisting of all p gene markers (dimension: N*p)
  * K = 2d array of floats consisting of the genetic relatedness of the N observations (dimension:N*N)

# Keyword arguments

  * addIntercept = Boolean flag indicating if an intercept column needs to be added to the design matrix; Default is to add an intercept
  * reml = Bool flag indicating if VCs are estimated by REML likelihood; Default is ML
  * method = String indicating the matrix factorization method to use; Default is QR.

# Value

A list of output values are returned:

  * out00.sigma2 = Float; estimated marginal variance due to random errors (by null lmm)
  * out00.h2 = Float; estimated heritability (by null lmm)
  * lod = 1d array of floats consisting of the lod scores of this trait and all markers (dimension: p*1)

# Some notes

```
This is a subsequent function that does univariate scan. For every scan for each genetic marker, the 
variance components will need to be re-estimated. To be called by the `scan` function when the `method = ` 
field is passed as `alt`.
```


In [13]:
@time test_alt = scan(pheno_y, geno, kinship; addIntercept = true, reml = false, assumption = "alt", method = "cholesky");

  0.548589 seconds (1.86 M allocations: 861.011 MiB, 26.28% gc time)


In [14]:
test_alt[1] # 

0.04983520992588347

In [15]:
test_alt[2]

0.8960637228416714

In [16]:
test_alt[3]

7321-element Vector{Float64}:
 0.8959152120195115
 0.8959152120195115
 0.8959152120195115
 0.8959152120195115
 0.8959152120195115
 0.8959152120196866
 0.8959152120195115
 0.8959152120195115
 0.8959152120195115
 0.8959152120195115
 0.8959152121140267
 0.8959152374339192
 0.8957761212977645
 ⋮
 0.9023536181416946
 0.9023536181056799
 0.9023536181056799
 0.8984874509594057
 0.8951424229522367
 0.8951424229522367
 0.8948313365595942
 0.8942228015686338
 0.8942884362217399
 0.893637575195784
 0.894429331508082
 0.894429331508082

In [17]:
summary_tab1 = DataFrame(Assumption = ["Null", "Alt"], 
                         Sigma2_e = [test_null[1], test_alt[1]], 
                         H2 = [test_null[2], test_alt[2]]);
print(summary_tab1)

2×3 DataFrame
 Row │ Assumption  Sigma2_e   H2       
     │ String      Float64    Float64  
─────┼─────────────────────────────────
   1 │ Null        0.0498352  0.896064
   2 │ Alt         0.0498352  0.896064

In [18]:
summary_tab1_lods = DataFrame(Marker = collect(1:10), Null = test_null[3][1:10], Alt = test_alt[4][1:10]);
print(summary_tab1_lods)

10×3 DataFrame
 Row │ Marker  Null        Alt        
     │ Int64   Float64     Float64    
─────┼────────────────────────────────
   1 │      1  0.00819636  0.00819776
   2 │      2  0.00819636  0.00819776
   3 │      3  0.00819636  0.00819776
   4 │      4  0.00819636  0.00819776
   5 │      5  0.00819636  0.00819776
   6 │      6  0.00819636  0.00819776
   7 │      7  0.00819636  0.00819776
   8 │      8  0.00819636  0.00819776
   9 │      9  0.00819636  0.00819776
  10 │     10  0.00819636  0.00819776

In [19]:
mean(test_null[3] .< test_alt[4])

1.0

## 2. Univariate Scans with permutation tests:

In [20]:
@time test_perms = scan_perms(pheno_y, geno, kinship; 
                              prior_a = 1.0, prior_b = 0.1,
                              addIntercept = true, reml = false, method = "cholesky",
                              nperms = 1000, original = true);

 13.576158 seconds (103.84 k allocations: 17.629 GiB, 13.04% gc time)


In [21]:
size(test_perms)

(1001, 7321)

In [22]:
@time test_perms_lite = scan_perms_lite(pheno_y, geno, kinship; 
                                   prior_a = 1.0, prior_b = 0.1,
                                   addIntercept = true, reml = false, method = "cholesky",
                                   nperms = 1000, original = true);
test_perms_lite = permutedims(test_perms_lite);

  0.717253 seconds (1.38 M allocations: 162.923 MiB, 82.84% compilation time)


In [23]:
sumSqDiff(test_perms, test_perms_lite)

8.861391813607934e-22